# PIR Génétique - UEG2: Génétique des populations et épidémiologie génétique

## **TP: Étude d'association pan-génomique dans le diabète de type 1**

*Claire Vandiedonck - 2022*

> 0. Synopsis
> 1. Première partie: Format de fichiers de génétique et découverte du logiciel PLINK
> 2. Deuxième partie: Statistiques descriptives et QC
> 3. Troisième partie: Analyse d'association génétique cas-contrôles
> 4. Quatrième partie: Analyse d'association familiale

## Avant d'aller plus loin
---
<div class="alert alert-block alert-danger"><b>Attention:</b> 
Ne travaillez pas directement sur ce notebook pour ne pas le perdre. Dupliquez-le et renommez-le par exemple en ajoutant vos initiales et travaillez sur cette nouvelle copie. Pour ce faire, dans le panneau de gauche, faites un clic droit sur le fichier et sélectionnez "Duplicate". Puis, toujours dans la colonne de gauche, faites un clic droit sur cette copie et sélectionnez "rename" pour changer le nom. Ouvrez ensuite cette nouvelle version en double cliquant dessus. Vous êtes prêt(e) à démarrer! <br>
<br>
<b>N'oubliez pas de sauvegarder régulièrement votre notebook</b>: <kbd>Ctrl</kbd> + <kbd>S</kbd>. ou en cliquant sur l'icone 💾 en haut à gauche de votre notebook ou dans le Menu du JupyterLab "File puis "Save Notebook"!
</div>

<div class="alert alert-block alert-info"> 
   
<b>Rappel :</b> Vous pouvez prendre des notes directement dans ce notebook, en ajoutant une cellule Markdown en cliquant sur l'icône <kbd>➕</kbd> dans la barre des menus, et en choisissant son format dans le menu déroulant. 
Les astuces pour utiliser les cellules des notebooks sont rappelées tout en bas dans un cadre bleu ⏬.
</div>

Dans ce TP, vous lancerez chaque cellule l'une après l'autre dans l'ordre. Si vous revenez en arrière et rééxécuter une cellule, veillez à relancer les suivantes. Les cellules sont numérotées dans leur ordre d'éxécution.

__*=> A propos de ce jupyter notebook*__

Pour ce TP, nous aurons besoin d'éxécuter du code **bash** et **R**. Nous allons utiliser un notebook avec un noyau **python** et indiquer que les cellules doivent être éxécutées en bash (natif) en ajoutant `%%bash` au début de chaque cellule, et en ajoutant `%%R` au début de chaque cellule `R` après avoir installé et chargé `Rpy2` au début du notebook. Cela restera très lisible.

Avant de commencer, nous devons donc charger le module `Rpy2` avec la commande suivante.

In [ ]:
# cell 1
%load_ext rpy2.ipython

Vous pouvez éxécuter cette commande python pour connaître votre répertoire de travail.

In [ ]:
# cell 2
import os 
os.getcwd()

<div class="alert alert-block alert-warning"><b>Le résultat devrait ressembler à :</b>"/srv/home/mylogin/pir_g2_gwas" avec votre login. Si ce n'est pas le cas, changez de répertoire avec la commande "os.chdir('path')" ou applez votre enseignant à l'aide!</div>

Pour lister ce qu'il contient, vous pouvez tenter avec les 3 langages:
- en python: la commande affichera aussi les fichiers cachés commençant par un `.`.

In [ ]:
# cell 3
os.listdir()

- en bash:

In [ ]:
%%bash
# cell 4
ls

- en R:

In [ ]:
%%R
# cell 5
list.files()


## **0 - Synopsis**
---
### Objectifs du TP

Dans ce TP, vous allez réaliser une étude d’association génétique dans le diabète de type 1 (DT1) dans une cohorte française au moyen d’outils informatiques et statistiques (Marees  AT et al., 2018).
Les sujets d'étude proviennent d'une série de patients atteints de diabète de type 1 et de contrôles sains. Tous les échantillons ont été génotypés au moyen de la puce Illumina immunochip (Trynka et al., 2011, Parkes M. et al., 2013). Au cours de ces séances de TP, vous serez amenés à:
- comprendre le format des fichiers de génotypage pour les logiciels d'analyse génétique
- effectuer des contrôles de qualité
- nettoyer/filtrer les données
- réaliser l'étude d'association pangénomique de type cas/contrôles et de type familial
- interpréter vos résultats en comparaison avec les articles publiés par le Type 1 Diabetes Genetics Consortium (Onengut-Gumuscu  S, et al., 2015, Robertson CC et al., 2021).

## **I. Première partie: format de fichiers de génétique et découverte du logiciel PLINK** *(~1 heure)*
---
    
### **I.A. Le logiciel PLINK**

Pour réaliser l'étude d'association, vous devrez préparer les fichiers sous un format utilisable par les logiciels de génétique. Le principal logiciel que vous utiliserez est **PLINK**: http://zzz.bwh.harvard.edu/plink/.

En plus de permettre des analyses d'association, ce logiciel permet de manier les fichiers de génétique, de faire des sélections et filtrages, et de réaliser certains contrôles de qualité. Il est mondialement reconnu comme un des meilleurs outils pour réaliser des GWAS.
Ce logiciel libre existe en versions windows, mac et unix.
Le tutoriel (http://zzz.bwh.harvard.edu/plink/tutorial.shtml) est très détaillé et donne de nombreux exemples.

Ce logiciel a été développé par Shaun Purcell. Il avait alors une double affiliation au Center for Human Genetic Research (CHGR), Massachusetts General Hospital (MGH) et au Broad Institute of Harvard & MIT.  Il est désormais chef de son labo (http://zzz.bwh.harvard.edu/index.html) basé au Brigham & Women's Hospital, affilié à la Harvard Medical School. La version originale actuelle présente sur son site web est la version v1.07 (10-Oct-2009).

Il existe à présent une nouvelle version beta **PLINK 1.9** dont l'objectif est principalement d'améliorer la vitesse et la fonctionnalité pour de très gros jeux de données. Ouvrez le lien "https://www.cog-genomics.org/plink2. Elle a été principalement développée par  par Christopher Chang at Human Longevity, Inc., par Carson Chow and Shashaank Vattikuti at the NIH-NIDDK's Laboratory of Biological Modeling, Laurent Tellier at the BGI Cognitive Genomics Lab, and James Lee at the University of Minnesota, with additional funding from the Purcell Lab at Mount Sinai School of Medicine du BGI Cognitive Genomics Lab! 

> Dans ce TP, vous utiliserez la version **PLINK 1.9** (https://www.cog-genomics.org/plink2) désormais stable (beta 6.24, 6 Jun 2021) qui a été installée sur adénine. Pour note, il existe une version PLINK2.0 en cours de développement.

<div class="alert alert-block alert-warning"><b>En pratique</b><br>PLINK s'utilise en lignes de commandes « relativement simples ». 
La plupart des commandes sont les mêmes que dans la version historique 1.07. Vous pouvez retrouver à ce lien <a href="http://zzz.bwh.harvard.edu/plink/reference.shtml"> </a>la liste de ces commandes.</div>

### **I.B. Lancement de PLINK**

Pour faire tourner PLINK sous Unix ou dans ce notebook, vous appelez d'abord le programme en tapant : `plink` dans une cellule **bash**. 

<span style="color:red"><i>N'ayez pas peur de la boite rouge qui apparait. Cet affichage disparaitra quand les commandes seront complètes.</i></span>

In [ ]:
%%bash
## cell 6
plink

Le programme démarre et s'arrête avec une erreur car vous n'avez pas encore donné de fichiers d'entrée (*inputs*).

> - Si vous travaillez **avec PLINK 1.07** au lieu de 1.9, PLINK va essayer de se connecter au réseau pour vérifier la version. Si vous voulez l'éviter, ajouter l'argument:`--noweb` avec la commande ```plink --noweb```
>
>- Si toutefois vous vouliez utiliser PLINK en dehors d'adenine, voici des conseils d'utilisation **sous Windows**:<br>
>Vous devez utiliser le DOS en utilisant l'invite de commande (cmd.exe) ou, sur les dernières versions de windows, le "windows power shell" à partir du menu "fichier" de votre répertoire de travail.  Vos fichiers "inputs" doivent être placés dans le même répertoire (=dossier) que l'éxécutable de PLINK plink.exe.
>Trois options s'offrent à vous, de la plus compliquée à la plus simple!:
>
>     **(1)** Dans le menu « Démarrer » de Windows, rechercher « cmd » puis indiquez le chemin du répertoire où est situé plink.exe (pour changer de répertoire, la commande est “cd”; pour passer au répertoire supérieur, taper “cd ..” ; vous pouvez utiliser la flèche de tabulation à gauche du clavier pour autocompléter les noms de dossiers)
> 
>     **(2)** Alternativement, trouvez le fichier cmd.exe (en général dans Windows\system 32; vous pouvez le trouver aussi dans les accessoires). Faire une copie que vous placez dans le dossier contenant plink.exe. Si vous double-cliquez alors sur cmd.exe dans ce répertoire, vous pouvez utiliser PLINK.
>
>     **(3)** Encore plus simplement, vous pouvez aussi créer en local un répertoire dans lequel vous mettrez tous vos documents du TP (fichiers d'input, d'output) ainsi que vos programmes. Copiez et collez l'executable plink.exe dans ce répertoire. Copiez et collez également l'exécutable de l'invite de commande cmd.exe dans ce répertoire. Si vous double-cliquez alors sur cmd.exe dans ce répertoire, vous pouvez utiliser PLINK.
>
>      Alternativement, si vous travaillez depuis windows power shell, il faut spécifier que plink est dans le répertoire de travail en tapant: .\plink
      

### **I.C. Fichiers d'entrée**

Commençons par nous familiariser avec l'utilisation avec des fichiers « test ».

PLINK a besoin de <u>deux types de fichiers d'entrée.</u>

- `.ped` pour le fichier de structure de pedigree
- `.map` pour le fichier des marqueurs génotypés

Dans le répertoire `input` de votre environnement jupyterlab, vous avez des fichiers d'entrée d'exemple avec le préfixe `test.`. Les fichiers sont aussi disponibles sur moodle.

##### **1. Exploration des fichiers d'entrée en dehors de PLINK**

<span style="color:red">Ouvrez-les en double cliquant dessus dans le jupyterlab </span> (ou avec un éditeur de texte comme notepad++ sous windows ou gedit sous unix si vous ne travaillez pas sur adénine).
Cherchez des explications sur leur structure dans la documentation de PLINK à ce lien "5. Basic usage/data formats" (http://zzz.bwh.harvard.edu/plink/data.shtml)  du menu.



<span style="color:blue"><b>Q1.1</b><i>  Concernant le fichier de structure de <b>pedigree</b>:</i></span>
    
<span style="color:blue"><i>- Combien de sujets ont été génotypés ?</i></span>

Votre réponse...

<span style="color:blue"><i>- Sont-ils apparentés ? Ont-ils les mêmes parents ? Quelles colonnes vous donnent ces informations ?</i></span>

Votre réponse...

<span style="color:blue"><i>- Quel est l'identifiant du 5ème sujet ? Est-il possible de le distinguer des autres sujets en considérant un identifiant unique ?</i></span>

Votre réponse...

<span style="color:blue"><i>- Quel est le sexe des sujets ? Quelle colonne vous donne cette information ? Est-ce cohérent avec les autres colonnes ?</i></span>

Votre réponse...

<span style="color:blue"><i>- Connaît-on leur statut vis-à-vis de la maladie ? Combien y a-t-il de cas et de contrôles ?</i></span>

Votre réponse...

<span style="color:blue"><i>- Combien de marqueurs ont été génotypés pour ces sujets ?</i></span>

Votre réponse...

<span style="color:blue"><i>- Combien de colonnes contiendraient un fichier de pedigree avec n marqueurs génotypés ?</i></span>

Votre réponse...

<span style="color:blue"><b>Q1.2.</b><i> Concernant le fichier de <b>marqueurs</b> :</i></span>
    
<span style="color:blue"><i>- Combien y-a-t-il de marqueurs dans ce fichier ? Est-ce en accord avec le fichier de pedigree ?</i></span>

Votre réponse...

<span style="color:blue"><i>- Sur quels chromosomes sont-ils situés ?</i></span>

Votre réponse...

<span style="color:blue"><i>- Connaissez-vous leur position physique ? A partir de quelle origine ?</i></span>

Votre réponse...

<span style="color:blue"><i>- Connaissez-vous leur position génétique ?</i></span>

Votre réponse...

##### **2.  Chargement des fichiers dans PLINK**

Nous allons à présent charger ces fichiers sans les modifier dans PLINK.<br> Il existe deux commandes possibles.

1. ```plink --file input/test```

ou

2. ```plink --ped input/test.ped --map input/test.map```

Dans la 1ère, nous utilisons l'argument `--file` qui nous permet de lire tous les fichiers inputs commençant par le préfixe `test` dans le répertoire `input`.
Dans la 2nde commande, nous utilisons un argument pour chaque type de fichier input. Cette seconce commande est plus longue à taper mais permet de charger des inputs qui n'auraient pas le même préfixe.

Exécutons ces 2 commandes l'une après l'autre. Elles retournent bien le même résultat. Une erreur apparaît dans les 2 cas. Elle était attendue. Nous reviendrons un peu plus bas sur cette erreur (partie I.E).

In [ ]:
%%bash
# cell 7
#la premiere commande
plink --file input/test

Notez que vous avez obtenu un fichier texte de sortie `plink.log` que vous pouvez ouvrir en double cliquant dessus. Il contient tout ce qui s'est affiché à la suite de la commande jusqu'à la ligne d'erreur.

In [ ]:
%%bash
# cell 8
#la deuxième commande
plink --ped input/test.ped --map input/test.map

Le fichier texte `plink.log` a été mis à jour et contient l'affichage des options/argument de cette 2ème commande. Voyons immédiatement comment  spécifier un nom aux fichiers de sortie pour ne pas les écraser à chaque utilisation de PLINK.

### **I.D. Spécification des noms de fichier de sortie**

A chaque commande PLINK, si vous spécifiez des arguments *(vrai aussi sans argument pour PLINK 1.07)*, des fichiers de sortie (*outputs*) sont automatiquement générés avec différentes extensions. En particulier, vous obtenez un fichier `.log` qui a sauvegardé tout ce qui s'était affiché dans votre terminal ou dans le notebook.

Vous pouvez spécifier le préfixe de ces outputs avec l'argument `--out`.

Dans PLINK, vous pouvez entrer plusieurs arguments à la suite sur la même ligne de commande. Si vous avez utilisé l'argument `-- out`, tous les autres outputs générés contiendront le même préfixe suivis de leur extension propre. On peut spécifier le répertoire dans lequel ils seront sauvergardés en indiquant le chemin.

On réessaye avec les 2 commandes précédentes en spécifiant un préfixe aux fichiers d'output que nous sauvegarderons dans le répertoire `output`

In [ ]:
%%bash
# cell 9
#la premiere commande avec la spécification de la sortie
plink --file input/test --out output/first

In [ ]:
%%bash
# cell 10
#la deuxième commande
plink --ped input/test.ped --map input/test.map --out output/second

Cette fois-ci, aucun fichier texte `plink.log` n'a été créé. Après la 1ère commande, un fichier `first.out` a été créé, après la seconde un fichier `second.out` a été créé. Ils sont dans le répertoire `output`. Vous pouvez les ouvrir. Notez qu'il contiennent désormais 2 lignes après « Options in effect ».

Un autre fichier a été créé après chaque commande avec l'extension `.fam`. Vous pouvez noter qu'ils contiennent le début du fichier `test.ped` présent dans `input`. Ils sont incomplets car la commande a été interrompue par l'erreur.

### **I.E. Correspondance des fichiers d'input**

<span style="color:blue"><b>Q1.3.</b><i> Quelle erreur avez-vous rencontrée avec la 1ère et la 2ème commande ?</i></span>

Votre réponse...

On apprend désormais que le fichier de pedigree ne contenait les génotypes que de marqueurs autosomiques. On a nettoyé le fichier afin de ne garder que les marqeurs autosomiques. Il a été sauvegardé sous un autre nom (`autosomic.map` déjà dans le répertoire `input`).

Chargez-le à présent dans PLINK avec le fichier de pedigree. On n'oublie-pas de changer le préfixe des sorties pour ne pas écraser le précédent fichier .log.

In [ ]:
%%bash
# cell 11
plink --ped input/test.ped --map input/autosomic.map --out output/autosomic

<span style="color:blue"><b>Q1.4.</b><i> Les informations obtenues sont-elles désormais cohérentes avec ce que vous savez?</i></span>

Votre réponse...


<div class="alert alert-block alert-warning"><b>En pratique: </b><br> Cette structure de pedigree et de liste de marqueurs est utilisée par la plupart des logiciels de génétique. Il existe cependant quelques variantes, en particulier pour l'encodage des phénotypes parfois dans un troisième fichier, ou pour le fichier de marqueurs qui ne contient pas toujours la position.

<b>L'ordre des colonnes des génotypes dans le fichier de pedigree doit toujours correspondre à l'ordre des marqueurs. Il est donc fondamental de bien formater ses fichiers !</b>
</div>

### **I.F. Fichiers binaires**

PLINK peut travailler avec des formats de fichier de grande taille : nombreux sujets et nombreux génotypes.
<br>
- Dans ce cas, il existe une **version binaire des fichiers** moins gourmande en espace disque et en temps d'analyse. Consultez la section « Binary PED files » (http://zzz.bwh.harvard.edu/plink/data.shtml#bed) et effectuez la conversion des fichiers test en format binaire en sauvegardant les outputs avec le préfixe `mytest_bdata` avec la commande suivante utilisant l'argument `--make-bed`.


In [ ]:
%%bash
#cell 12
plink --ped input/test.ped --map input/autosomic.map --make-bed --out output/mytest_bdata

<span style="color:blue"><b>Q1.5.</b><i>Concernant ce <b>format binaire</b> :</i></span>

<span style="color:blue"><i>- Quel fichier est converti en format réellement binaire ? Quelle est l'extension de ce fichier ?</i></span>

Votre réponse...

<span style="color:blue"><i>- Quels autres fichiers ont été générés ? A quoi correspondent-ils ?</i></span>

Votre réponse...

<span style="color:blue"><i>- Quelle information n'était pas dans le fichier .map et est désormais présente dans le fichier de marqueurs ?</i></span>

Votre réponse...

**- Pour lire avec PLINK les fichiers binaires** ainsi générés, on utilise l'argument `--bfile` comme dans la commande suivante (qui s'interrompt car aucune manipulation ni output spécifique en dehors du .log n'a été demandé):

In [ ]:
%%bash
# cell 13
plink --bfile output/mytest_bdata --out output/load_mytest_bdata

- Inversement, il existe un moyen de **convertir des données binaires en format non binaire** (cf. aide la section 6. Data management: http://zzz.bwh.harvard.edu/plink/dataman.shtml) avec l'argument `--recode`

In [ ]:
%%bash
# cell 14
plink --bfile output/mytest_bdata --recode --out output/mytest_data

<span style="color:red"><i>Ne faites pas attention aux chiffres affichés dans la ligne commençant par "Calculating allele frequencies" si vous lisez le log directement sur le notebook. Cette même ligne a une apparence normale sans ces chiffres si vous lisez le fichier d'output `.log`.</i></span>

<span style="color:blue"><b>Q1.6.</b><i>Quelle différence observez vous entre le <b>nouveau fichier</b> `.ped` et le fichier `.ped` <b>initial</b> ?</i></span>

Votre réponse...

<div class="alert alert-block alert-warning"><b>En résumé</b><br>
Vous avez appris à:<br>
    - identifier les commandes de base dans PLINK pour lire les fichiers d'inputs<br>
    - ajouter un suffixe aux fichiers d'outputs<br>
    - convertir les fichiers entre les formats plats et binaires.<br>
    
Le rappel de ces commandes, tiré de la liste des commandes PLINK disponibles à <a href="http://zzz.bwh.harvard.edu/plink/reference.shtml">ce lien</a>, est résumé ci-dessous:<br>
    

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-0pky">Option</th>
    <th class="tg-0pky">Parameter/default</th>
    <th class="tg-0pky">Description</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0pky">Basic input/output</td>
    <td class="tg-0pky"></td>
    <td class="tg-0pky"></td>
  </tr>
  <tr>
    <td class="tg-0pky">--file</td>
    <td class="tg-0pky"> {plink}&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
    <td class="tg-0pky">&nbsp;&nbsp;Specify .ped and .map files</td>
  </tr>
  <tr>
    <td class="tg-0pky">--ped</td>
    <td class="tg-0pky">&nbsp;&nbsp;{plink.ped}&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
    <td class="tg-0pky">&nbsp;&nbsp;Specify .ped file</td>
  </tr>
  <tr>
    <td class="tg-0pky">--map</td>
    <td class="tg-0pky">&nbsp;&nbsp;{plink.map}&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
    <td class="tg-0pky">&nbsp;&nbsp;Specify .map file</td>
  </tr>
  <tr>
    <td class="tg-0pky">--bfile</td>
    <td class="tg-0pky"> {plink}&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
    <td class="tg-0pky">&nbsp;&nbsp;Specify .bed, .bim and .fam</td>
  </tr>
  <tr>
    <td class="tg-0pky">--bed</td>
    <td class="tg-0pky"> {plink.bed}&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
    <td class="tg-0pky">&nbsp;&nbsp;Specify .bed file</td>
  </tr>
  <tr>
    <td class="tg-0pky">--bim</td>
    <td class="tg-0pky"> {plink.bim}&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
    <td class="tg-0pky">&nbsp;&nbsp;Specify .bim file</td>
  </tr>
  <tr>
    <td class="tg-0pky">--fam</td>
    <td class="tg-0pky"> {plink.fam}&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
    <td class="tg-0pky">&nbsp;&nbsp;Specify .fam file</td>
  </tr>
  <tr>
    <td class="tg-0pky">--out</td>
    <td class="tg-0pky"> {plink}&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
    <td class="tg-0pky">&nbsp;&nbsp;Specify output root filename</td>
  </tr>
  <tr>
    <td class="tg-0pky">Other data management options</td>
    <td class="tg-0pky"></td>
    <td class="tg-0pky"></td>
  </tr>
  <tr>
    <td class="tg-0pky">--make-bed</td>
    <td class="tg-0pky">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
    <td class="tg-0pky">&nbsp;&nbsp;Make .bed, .fam and .bim</td>
  </tr>
  <tr>
    <td class="tg-0pky">--recode</td>
    <td class="tg-0pky">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
    <td class="tg-0pky">&nbsp;&nbsp;Output new .ped and .map files</td>
  </tr>
</tbody>
</table>
</div>

<div class="alert alert-block alert-success"><b>=> Bravo !</b><br>Vous avez fait un bel effort pour apprendre à utiliser PLINK avec les commandes de base. Vous êtes désormais prêts à travailler avec de vrais fichiers de patients (les cas) et de contrôles. Nettoyez juste avant le dossier « output » de ces fichiers d'entrainement (rm = remove) avec la commande suivante:</div>

In [ ]:
%%bash
# cell 15
rm output/*

## **II. Deuxième partie: Statistiques descriptives et QC** *(~ 45')*
---
    

### **II.A. Lecture des fichiers du jeu de données de cas et contrôles**

Les jeux de données des cas et des témoins ont été préalablement formattés et fusionnés pour obtenir un dataset unique directement utilisable par PLINK. 

 Les fichiers ont les préfixes `t1dcc` (pour T1D Case Control). Les fichiers sont dans le répertoire `input` de votre environnement sauf le fichier `t1dcc.bed` qui était trop volumineux pour être préinstallé.
 Nous allons le copier dans votre répertoire `input`.

In [ ]:
%%bash
# cell 16
cp /srv/data/pir-g2/input/t1dcc.bed ./input

Avec la version PLINK 1.9, il n'est pas possible de lire les fichiers d'inputs sans exécuter une commande sur ces fichiers. Utilisons-en une toute simple qui écrit la liste des SNPs `--write-snplist` sans modifier nos fichiers d'inputs.

In [ ]:
%%bash
# cell 17
plink --bfile input/t1dcc --write-snplist --out output/read_t1dcc

<span style="color:blue"><b>Q2.1.</b><i> Description des fichiers et données:</i></span>

<span style="color:blue"><i>- Quel est le format des fichiers d'entrée? </i></span>

Votre réponse...

<span style="color:blue"><i>- Donnez le nombre de sujets? </i></span>
 

Votre réponse...

<span style="color:blue"><i>- Comment se répartissent-ils entre patients et contrôles? </i></span>

Votre réponse...

<span style="color:blue"><i>- Indiquer les effectifs par sexe.</i></span>

Votre réponse...

<span style="color:blue"><i>- Combien de marqueurs sont inclus ? </i></span>

Votre réponse...

<span style="color:blue"><i>- Quel est le taux moyen de génotypage par SNP? </i></span>

Votre réponse...

### **II.B. Contrôle de qualité**

Notre jeu de données ainsi prêt, vous pouvez procéder aux analyses de contrôle de qualité avec plusieurs étapes de filtres pour élminier les données de mauvaise qualité.<br>
Il est recommandé de commencer par les contrôles de qualité sur les SNPs, avant de procéder aux contrôles de qualité sur les sujets.

<div class="alert alert-block alert-danger"><b>Attention: à chaque étape de nettoyage, nous donnons un nouveau nom aux fichiers!</b> 
<br>=> Si vous voulez générer des fichiers .ped et .map, la commande est : <b>--recode</b>
<br>=> Si vous voulez générer des fichiers .bed, .bim et .fam, la commande est : <b>--make-bed</b>
</div>

##### **-> QC sur les SNPs**

- Exclusion des SNPs avec un **taux de génotypage <95 %** dans l'ensemble de l'étude

In [ ]:
%%bash
# cell 18
plink --bfile input/t1dcc --geno 0.05 --make-bed --out output/filter1

- Exclusion des SNPs avec des **taux de génotypage trop différents entre patients et contrôles**

In [ ]:
%%bash
# cell 19
plink --bfile output/filter1 --test-missing --pfilter 0.00001 --out output/missingcc

Cette 1ère commande a généré la liste des SNPs problématiques dans un fichier « missingcc.missing ». Excluez-les à présent avec cette 2nde commande  :

In [ ]:
%%bash
# cell 20
plink --bfile output/filter1 --exclude output/missingcc.missing --make-bed --out output/filter2

- Exclusion des **SNPs qui ne sont pas en équilibre de Hardy-Weinberg** chez les contrôles au seuil 10<sup>-4</sup>

In [ ]:
%%bash
# cell 21
plink --bfile output/filter2 --hwe 0.0001 --make-bed --out output/filter3

- Exclusion des SNPs rares avec une **MAF < 0.01**

In [ ]:
%%bash
# cell 22
plink --bfile output/filter3 --maf 0.01 --make-bed --out output/filter4

D'autres contrôles peuvent être effectués : vérifier que les SNPs ne sont pas dupliqués (PLINK le signale automatiquement), que les SNPs n'ont qu'une seule position génomique sur le génome, etc...

<span style="color:blue"><b>Q2.2.</b><i>Combien de marqueurs sont exclus à chaque étape? Combien reste-t-il de marqueurs après ce QC sur les SNPs ?</i></span>

Votre réponse...

<span style="color:blue"><b>Q2.3.</b><i> Pourquoi ne considère-t-on pas les SNPs rares dans cette étude ?</i></span>

Votre réponse...

##### **->QC sur les sujets**

- Exclusion des sujets avec un **taux de génotypage <95 %**

In [ ]:
%%bash
# cell 23
plink --bfile output/filter4 --mind 0.05 --make-bed --out output/filter5

- Exclusion des **sujets apparentés**

La détection des sujets apparentés peut se faire avec PLINK mais nécessite de nombreuses commandes (`--genome`, ) et est très longue à tourner.
<br>Il est plus aisé d'utiliser un logiciel dédié à cette analyse : **KING** (Kinship-based Infererence for GWAS): http://people.virginia.edu/~wc9c/KING/ . Ce logiciel est très rapide, permet de lire des fichiers binaires de PLINK. Il permet aussi d'analyser la structure de la population.
> Si vous utiliez KING en dehors d'adénine, téléchargez-le, dézippez le et copier une cmd.exe dans le même répertoire si vous travaillez sous windows ou copiez le king.exe dans votre répertoire de travail. Comme pour PLINK, les fichiers d'entrée doivent être dans le même répertoire.

Exécutez la commande suivante (<mark>elle prend quelques secondes</mark>). Comme **KING** ne génère pas de fichier `.log`, nous n'avons pas à nous inquiéter d'écraser le log précédent.

In [ ]:
%%bash
# cell 24
king -b output/filter5.bed --kinship --related --prefix output/filter5

Cette commade vous a gènèré un fichier `filter5.kin` indiquant les relations dans les familles, et un fichier `filter5.kin0` indiquant les relations entre les familles. Dans ces fichiers, la colonne « Kinship » correspond au coefficient de corrélation estimé. S'il est > 0.17, ils est probable qu'ils soient apparentés au moins au 1er degré.

La commande suivante vous permet de récupérer la liste des sujets non apparentés (<mark>elle prend quelques secondes, soyez patients!</mark>). Attention elle s'effectue toujours sur l'input de prefixe `filter5` car la commande précédente de **KING** n'a pas généré de nouveaux fichiers de pedigree. 

In [ ]:
%%bash
# cell 25
king -b output/filter5.bed --kinship --unrelated --prefix output/filter5

La commande suivante de **PLINK** vous permet de sélectionner les sujets non apparentés :

In [ ]:
%%bash
# cell 26
plink --bfile output/filter5 --keep output/filter5unrelated.txt --make-bed --out output/filter6

De nombreux autres vérifications peuvent être effectuées :
<br>On peut par exemple vérifier que les génotypes sur les chromosomes X et Y, s'ils sont disponibles, sont en accord avec le sexe des sujets. On peut également vérifier que les sujets ne sont pas en excès d'hétérozygotie, ce qui pourrait indiquer une contamination par un autre ADN. Si certains sujets ont été génotypés plusieurs fois (contrôle positif présent sur chaque plaque de génotypage), il est bon de vérifier que les génotypes sont identiques. De même, on peut avoir placé des sujets de génotype déjà connus pour tous les SNPs ou certains SNPs de la puce et vérifier la concordance.

<span style="color:blue"><b>Q2.4.</b><i> Combien de sujets sont exclus à chaque étape? Combien reste-t-il de cas et de contrôles à l'issue de ce QC ?</i></span>

Votre réponse...

<span style="color:blue"><b>Q2.5.</b><i> Quel contrôle essentiel pour une étude d'association cas-contrôle n'a-t-on pas encore fait ?</i></span>

Votre réponse...

Nous le ferons plus tard, après une première tentative de GWAS.

## **III. Troisième partie: Analyse d'association génétique cas-contrôles** *(~ 45')* 
---

### **III.A. 1er essai de test d'association allélique pangénomique**

**-> test de Chi2**

Effectuez maintenant le test d'association allélique pangénomique avec les arguments:<br>
```--assoc```: un test de Chi2<br>
```--adjust```: génère les pvalues ajustées également<br>
```--ci 0.95```: afin d'avoir l'IC à 95% de l'OR

In [ ]:
%%bash
# cell 27
plink --bfile output/filter6 --assoc --ci 0.95 --adjust --out output/allelictest

<span style="color:blue"><b>Q3.1.</b><i> Quels sont les fichiers de sortie produits par cette commande ? A quoi correspondent-ils ?</i></span>

Votre réponse...

<div class="alert alert-block alert-warning"><b>Au lieu de faire un test de Chi2 qui repose sur une approximation des distributions des allèles selon le statut, il est préférable de réaliser un test exact de Fisher qui prend en compte la distribution réelle de vos données.</b> Ce test peut-être très long à réaliser. Mais avec les outils informatiques, c'est désormais plus accessible. Nous allons donc refaire cette même étude d'association avec le test exact de Fisher. Au lieu de taper l'option `--assoc` il faut taper l'option `--fisher`. Les deux fichiers de résultats obtenus contiendront par défaut le mot "fisher". Il serait donc répétitif de le mettre dans le préfixe de vos fichiers de sortie que je vous propose de simplifier par "allelic".</div>

**-> test exact de Fisher**

In [ ]:
%%bash
# cell 28
plink --bfile output/filter6 --fisher --ci 0.95 --adjust --out output/allelic

-> Vous voyez, ca a été rapide avec PLINK1.9 et adenine!

Par défaut PLINK fournit des outputs avec des espaces multiples comme séparateur de colonnes, afin de les aligner visuellement. Mais pour certaines autres applications, il peut être utile de le convertir en format avec des tabulations comme séparateur de colonne. Dans la documentation de PLINK 1.9 à ce lien https://www.cog-genomics.org/plink/1.9/other, ils expliquent comment y parvenir avec trois commandes que nous enchaînons avec des pipes:
- une première commande avec `sed` pour remplacer l'espace en début de chaque ligne par rien
- une seconde commande avec `sed` pour remplacer l'espace en fin de chaque ligne par rien
- une dernière commande avec `tr` pour remplacer (translate) les séries de 1 à plusieurs espaces en une seule tabulation
Pour pouvoir ouvrir ce fichier directement avec un outil de tabulation (excel, librecalc ou TSVTable sur le jupyterLab), nous sauvegardons ce nouveau fichier en ajoutant l'extension `.tsv`.

Nous appliquons ces commandes ci-dessous pour les deux fichiers de sortie principaux de plink:
- allelic.assoc.fisher trié par position physique
- allelic.assoc.fisher.adjusted trié par pvalue croissante

In [ ]:
%%bash
# cell 29
cat output/allelic.assoc.fisher | sed 's/^[[:space:]]*//g' | sed 's/[[:space:]]*$//g' | tr -s ' ' '\t' > output/allelic.assoc.fisher.tsv
cat output/allelic.assoc.fisher.adjusted | sed 's/^[[:space:]]*//g' | sed 's/[[:space:]]*$//g' | tr -s ' ' '\t' > output/allelic.assoc.fisher.adjusted.tsv

**-> visualisation des résultats et QC**

A présent, nous allons explorer visuellement les résultats du test exact de Fisher en utilisant le paquet **qqman** de **R** (https://cran.r-project.org/package=qqman).

Comme lorsque l'on lançait des commandes bash, il est nécessaire dans ce notebook avec un noyau python, de spécifier que la cellule sera exéutée dans un autre langage R que python. Pour ce faire, <mark>après avoir bien lancé la 1ère cellule de ce notebook qui active le module `Rpy2`</mark>, il suffit alors d'écrire `%%R` sur la 1ère ligne de la cellule. Dans un notebook avec un noyau R, ce ne serait pas nécessaire.

Chargeons donc le paquet R depuis votre librairie et vérifions. En principe, vous avez la version `0.1.4` de qqman qui a déjà été installée dans votre environnement JupyterLab.

In [ ]:
%%R
# cell 30
library(qqman)
sessionInfo()

Pour savoir comment utiliser qqman, vous pouvez consulter la vignette de ce paquet au lien suivant: https://cran.r-project.org/web/packages/qqman/vignettes/qqman.html

- Nous allons commencer par charger dans R les résultats de l'étude d'association allélique obtenue avec PLINK avec la fonction `read.table()`.

In [ ]:
%%R
# cell 31
gwasResults <- read.table("output/allelic.assoc.fisher",header=T)# on peut charger directement dans R la version avec les espaces comme séparateur de champs!
str(gwasResults)# pour afficher la strture de l'objet
head(gwasResults)# pour afficher les 6 premières lignes

- Tracez ensuite votre Manhattan plot avec la fonction `manhattan()` avec les commandes suivantes:

In [ ]:
%%R
# cell 32
manhattan(gwasResults)# une première fois pour qu'il s'affiche dans le notebook
png("output/Fisher_test_manhattan.png")# on recommence pour sauvegarder la figure à part
manhattan(gwasResults)
dev.off()

- Il est également recommandé de tracer un qqplot, qui visualise la conformité de la distribution des p-values à celle attendue selon une loi uniforme en cas d'absence d'association (hypothèse nulle). Le même paquet **qqman** vous permet de tracer ce qqplot avec la fonction `qq()`.

In [ ]:
%%R
# cell 33
qq(gwasResults$P, main="Q-Qplot of P-values")# une première fois pour qu'il s'affiche dans le notebook

png("output/Fisher_test_qqplot.png")# on recommence pour sauvegarder la figure à part
qq(gwasResults$P, main="Q-Qplot of exact Fisher test P-values")
dev.off()

- On réajuste les axes pour avoir la ligne rouge en diagonale

In [ ]:
%%R
# cell 34
qq(gwasResults$P, main="Q-Qplot of P-values",xlim=c(0,max(-log10(gwasResults$P))))

<span style="color:blue"><b>Q3.2. </b><i>Que pensez-vous de ces résultats à l'échelle pangénomique ?</i></span>

Votre réponse...

<span style="color:blue"><b>Q3.3. </b><i>Dans le fichier « allelic.log » généré par PLINK, quelle valeur d'inflation factor obtenez-vous ? Une valeur proche de 1 indique une population homogène. Qu'en concluez-vous sur votre échantillon et les résultats obtenus ?</i></span>

Votre réponse...

### **III.B. GWAS en prenant compte de la stratification de population**

#### **1. Evaluation de la stratification de population**

Pour évaluer la stratification de population, vous réaliserez une ***MDS*** = multi-dimensional-scaling analysis avec le logiciel **KING** (https://people.virginia.edu/~wc9c/KING/kingpopulation.html). Ce type d'analyse est similaire à une ACP, mais au lieu d'utiliser les covariances entre les variables, la MDS prend en compte la distance Euclidienne et est plus adaptée pour rapprocher les individus.

- Lancez la commande suivante pour faire une **analyse MDS** (cette commande peut prendre quelques secondes):

In [ ]:
%%bash
# cell 35
king -b output/filter6.bed --mds --pcs 20 --prefix output/stratif

-> Regardez le fichier `stratifpc.txt`. Les colonnes 7 à 26 correspondent aux 20 premiers vecteurs de cette analyse MDS (10 par défaut que nous avons modifié à 20 avec l'option `--pcs`).

- A présent, dans R, nous allons **visualiser cette analyse MDS** en séparant artificiellement la représentation des contrôles (les 357 premiers dans le fichier) et les contrôles (de 358 à 572), pour mieux visualiser les différences.

In [ ]:
%%R
# cell 36
mds <- read.table("output/stratifpc.txt", header=T, stringsAsFactors=FALSE)
#png("output/mds.png")
par(mfrow=c(2,2))
plot(mds$PC1, mds$PC2, type="n", xlab="PC1", ylab="PC2", main="MDS PC1-PC2 in Controls")
points(mds$PC1[1:357], mds$PC2[1:357], col="black")
plot(mds$PC1, mds$PC2, type="n",xlab="PC1", ylab="PC2", main="MDS PC1-PC2 2 in Cases" )
points(mds$PC1[358:572], mds$PC2[358:572], col="red")
plot(mds$PC2, mds$PC3, type="n", xlab="PC2", ylab="PC3", main="MDS PC2-PC3 in Controls")
points(mds$PC2[1:357], mds$PC3[1:357], col="black")
plot(mds$PC2, mds$PC3, type="n", xlab="PC2", ylab="PC3", main="MDS PC2-PC3 3 in Cases")
points(mds$PC2[358:572], mds$PC3[358:572], col="red")
#dev.off

<span style="color:blue"><b>Q3.4. </b><i>Que pensez-vous de ces graphiques ?</i></span>

Votre réponse...

#### **2. GWAS corrigé pour la stratification de population**

Nous allons utiliser les vecteurs propres de l'analyse MDS comme covariable dans l'étude d'association en utilisant un modèle logistique dans PLINK sous un modèle additif.

La prise en compte des 20 composantes de la MDS peut prendre jusqu'à 1h30 sur des ordinateurs classiques. Sur adenine, elle devrait prendre <mark>2 à 5 minutes max</mark>. Le fichier `stratif.txt` est lu comme un fichier "phenotype" de PLINK avec chaque colonne 3 à n lue comme une covariable. Les vecteurs de l'analyses MDS correspondent seraient ici les covariables 5 à 24 pour PLINK avec l'argument `--covar-number`. Il est plus simple de donner leur nom avec l'argument `--covar-name` dans la commande ci-dessous.
> Si toutefois vous vouliez tester le TP sur une autre machine qu'adenine, nous vous recommandons de ne prendre en compte que les 2 ou 5 premières composantes, ce qui prend 5 à 15 minutes seulement. Changez juste le numéro des covariables dans la commande.

In [ ]:
%%bash
# cell 37
plink --bfile output/filter6 --logistic --adjust --ci 0.95 --covar output/stratifpc.txt --covar-name PC1-PC20 --hide-covar --out output/aftermds_PC20

- De nouveau, nous **sauvegardons une version `.tsv` des deux fichiers de sortie**:  `.logistic` trié par position physique, et `.logistic.adjusted` trié par pvalue.

In [ ]:
%%bash
# cell 38
cat output/aftermds_PC20.assoc.logistic | sed 's/^[[:space:]]*//g' | sed 's/[[:space:]]*$//g' | tr -s ' ' '\t' > output/aftermds_PC20.assoc.logistic.tsv
cat output/aftermds_PC20.assoc.logistic.adjusted | sed 's/^[[:space:]]*//g' | sed 's/[[:space:]]*$//g' | tr -s ' ' '\t' > output/aftermds_PC20.assoc.logistic.adjusted.tsv

- De nouveau, **visualisez vos résultats** avec un Manhattan plot et un qq-plot dans R:

In [ ]:
%%R
# cell 39
library(qqman)
gwasResultsAfterMDS <- read.table("output/aftermds_PC20.assoc.logistic", header=T)
#png("output/gwas_after_stratif.png")
par(mfrow=c(2,1))
manhattan(gwasResultsAfterMDS[! is.na(gwasResultsAfterMDS$P),])
qq(gwasResultsAfterMDS$P, main="Q-Qplot of P-values")
#dev.off()

<span style="color:blue"><b>Q3.5. </b><i>La prise en compte de la stratification vous semble-t-elle efficace ? Complètement efficace?</i></span>

Votre réponse...

<span style="color:blue"><b>Q3.6. </b><i>Comparez les régions associées à celles déjà connues en vous référant aux articles (Onengut-Gumuscu S, et al., 2015, Robertson CC et al., 2021; présents dans l'environnement).</i></span>

Votre réponse...

Une des difficulté de la puce immunochip est qu'elle ne contient pas de SNPs de référence "neutres" pour les études de population. Elle est au contraire enrichie en SNPs dans des régions du génome impliquées dans la réponse immunitaire, donc soumis à sélection.


<span style="color:blue"><b>Q3.7. </b><i>Quel type d'étude d'association permettrait de surmonter cette difficulté?</i></span>

Votre réponse...

> ***Pour aller plus loin:***
>Si vous les souhaitez, vous pouvez réaliser le même test avec un autre modèle que le modèle additif en ajoutant les arguments: 	`--dominant / --recessive / -- genotypic`<br>
>Vous pouvez utiliser un autre logiciel, GCTA  pour Genome-wide Complex Trait Analysis  (http://cnsgenomics.com/software/gcta/index.html) de plus en plus populaire en génétique multifactorielle, en particulier pour estimer la variance phénotypique expliquée par les différentes régions associées.

## **IV - Quatrième partie: Étude d'association familiale** *(~ 20-30')*

---
Vous allez étendre l'étude d'association de type cas-contrôles réalisée précédemment à une cohorte familiale de patients atteints de diabète de type 1. En fait, pour la plupart des patients étudiés dans l'étude d'association cas-contrôles ci-dessus, leurs parents sains ont également été collectés et génotypés avec la puce immunochip.

Les données concernant les autosomes ont déjà été formatées pour être utilisées dans PLINK.
Plusieurs contrôles qualité ont été effectués en amont:

- erreurs mendéliennes identifiées: retrait des familles ou individus problématiques si plusieurs marqueurs sont concernés
- SNPs en déséquilibre de Hardy-Weinberg éliminés
- SNPs avec une MAF < 0.01 éliminés
- SNPs et individus avec taux de génotypage insuffisant (<95%) éliminés 

Les données ainsi nettoyées pour ces familles nucléaires (ou trios) sont disponibles sur moodle avec le préfixe `ic_trio`. 

- Faites tourner l'étude d'**association familiale de type TDT** avec l'argument `--tdt` dans la commande ci-dessous:

In [ ]:
%%bash
# cell 40
plink --bfile input/ic_trio --tdt --adjust --ci 0.95 --out output/tdttrio

<span style="color:blue"><b>Q4.1. </b><i>Identifier les informations descriptives relatives aux SNPs et aux sujets inclus?</i></span>

Votre réponse...

- Ouvrez le fichier de sortie `tdttrio.tdt` qui recence les résultats ordonnés par position génomique. Dans cette analyse, considérez la colonne P pour connaitre le niveau de signification statistique non corrigé. Ouvrez aussi le fichier `tdttrio.tdt.adjusted` qui contient les résultats triés en fonction du niveau de signification croissant.

Nous en sauvegardons également une version `.tsv` au cas où.

In [ ]:
%%bash
# cell 41
cat output/tdttrio.tdt | sed 's/^[[:space:]]*//g' | sed 's/[[:space:]]*$//g' | tr -s ' ' '\t' > output/tdttrio.tdt.tsv
cat output/tdttrio.tdt.adjusted | sed 's/^[[:space:]]*//g' | sed 's/[[:space:]]*$//g' | tr -s ' ' '\t' > output/tdttrio.tdt.adjusted.tsv

- Utiliser qqman sous R pour tracer le **qqplot et le manhattan plot** que vous pourrez sauvegarder.

In [ ]:
%%R
# cell 42
tdt <- read.table("output/tdttrio.tdt", header=T)
head(tdt)

In [ ]:
%%R
# cell 43
#png("output/manhattan_trios.png")
manhattan(tdt[! is.na(tdt$P),], p="P")
#dev.off()
#png("output/qqplot_trios.png")
qq(tdt$P)
#dev.off()

<span style="color:blue"><b>Q4.2. </b><i>Que pensez-vous de ces résultats ?</i></span> 

Votre réponse...

<span style="color:blue"><b>Q4.3. </b><i>En dehors du CMH (chr6 entre 27 et 33 Mb), combien de SNPs ou de régions sont associés de manière significative (5x10<sup>-8</sup>)? Combien de SNPs ou de régions sont associés de manière suggestive (10<sup>-5</sup>)?</i></span> 

Votre réponse...

<span style="color:blue"><b>Q4.4. </b><i>Les SNPs associés à la maladie et les régions correspondantes sont-ils les mêmes que ceux de l'étude cas-contrôles?</i></span>

Votre réponse...

<span style="color:blue"><b>Q4.5. </b><i>Les SNPs associés à la maladie et les régions correspondantes sont-ils les mêmes que ceux déjà connus dans la littérature? </i></span>

Votre réponse...

Vous pourriez aussi explorer si les SNPs associés à la maladie, et les régions correspondantes, ont déjà été associés à d'autres maladies ou traits. Pour les autres maladies ou traits, vous pouvez consulter le GWAS catalogue https://www.ebi.ac.uk/gwas/, ClinVar https://www.ncbi.nlm.nih.gov/clinvar/, ou  les bases de données de variants comme dbSNP (https://www.ncbi.nlm.nih.gov/snp/ ou de gènes comme Gene du ncbi https://www.ncbi.nlm.nih.gov/gene/ ou Genecards https://www.genecards.org/ voire d'eQTLs comme GTEx https://gtexportal.org/home/. Vous pouvez aussi voir s'ils sont en DL avec d'autres SNPs qui eux avec la suite LDlink https://ldlink.nci.nih.gov/?tab=home.

<div class="alert alert-block alert-success"><b>=> Bravo!</b><br>

Vous avez réussi à mener votre 1er GWAS de bout en bout!</div>

<div class="alert alert-block alert-warning"><b>Pour aller plus loin: </b>
    
- Le logiciel <b>LocusZoom</b> (http://locuszoom.sph.umich.edu/locuszoom/genform.php?type=yourdata) vous permet de réaliser un zoom du manhattan plot centré sur un SNP ou un gène donné en ligne en renseignant le fichier de sortie de PLINK. La figure est générée dans un format .pdf.
Pour ce faire:<br>
i. télécharger le fichier de résultats de l'étude d'association trié par position physique<br>
ii. dans la 1ère boîte de locuzoom, cliquez à droite sur "PLINK data" en bleu -> cela spécificiera le bon format<br>
iii. toujours dnas la 1ère boîte, dans "Path to Your File": choisissez votre fichier de résultat<br>
iv. dans la seconde boîte, sélectionnez soit le SNP d'intérêt, soit un gène soit une région<br>
v. dans les options, sélectionnez un panel pertinent pour afficher le DL de la région.<br>

<mark><b>Attention:</b> notre jeu de données est sur la version <b>hg19/GRCHh37</b> du génome</mark>

Vous pouvez utiliser cet outil en ligne pour la ou les région(s) associée(s).
Vous pouvez aussi estimer le déséquilibre de liaison local avec PLINK et consulter le DL rapporté dans les populations de référence, par exemple en consultant par exemple les outils de la suite DLink https://ldlink.nci.nih.gov/?tab=home.    
    
    
- D'autres logiciels performants vous permettent d'éffectuer des <b>analyses d'association familiales</b>: UNPHASED (https://sites.google.com/site/fdudbridge/software/unphased-3-1) et BEAGLE (http://faculty.washington.edu/browning/beagle/beagle.html).<br><br>
    
- Vous pouvez également faire des analyses combinant plusieurs SNPs ou conditionnées sur certains SNPs.
</div>

***Références***<br>
1. Trynka G, et al. Dense genotyping identifies and localizes multiple common and rare variant association signals in celiac disease. Nature Genetics 43, 1193–1201 (2011) doi.org/10.1038/ng.998<br>
2. Parkes M, et al. Genetic insight into common pathways and complex relationships among immune-mediated diseases. Nature Genetics 14, 661-673 (2013) doi.org/10.1038/nrg3502<br>
3. Marees AT et al. A tutorial on conducting genome‐wide association studies: Quality control and statistical analysis. Int J Methods Psychiatr Res. 27:e1608. (2018) doi.org/10.1002/mpr.1608<br>
4. Onengut-Gumuscu S et al. Fine mapping of type 1 diabetes susceptibility loci and evidence for colocalization of causal variants with lymphoid gene enhancers. Nature Genetics 47, 381–386 (2015) doi.org/10.1038/ng.3245<br>
5. Robertson CC et al. (2021). Fine-mapping, trans-ancestral and genomic analyses identify causal variants, cells, genes and drug targets for type 1 diabetes. Nature Genetics 53, 962–971 (2021). doi.org/10.1038/s41588-021-00880-5

***Useful links***    
*To make nice html reports with markdown:* [html visualization tool1](https://dillinger.io/) or [html visualization tool2](https://stackedit.io/app#) and [to draw nice tables](https://www.tablesgenerator.com/markdown_tables ) and the [Ultimate guide](https://medium.com/analytics-vidhya/the-ultimate-markdown-guide-for-jupyter-notebook-d5e5abf728fd)   
*Further reading on JupyterLab notebooks:*  https://jupyterlab.readthedocs.io/en/latest/user/notebook.html <br>
*Here we are using JupyterLab interface implemented as part of the https://plasmabio.org/ project lead by Sandrine Caburet, Pierre Poulain and Claire Vandiedonck.*

Les informations de configuration de ce notebook et de l'environnement jupyterlab associé sont disponibles à ce lien: https://github.com/CVandiedonck/T1D_GWAS_short_version

---

<div class="alert alert-block alert-info"> 
    
<b>Rappel</b>, dans un notebook :

- la combinaison de touches <kbd>Ctrl</kbd>+<kbd>Entrée</kbd> exécute une cellule.<br>
- la combinaison de touches <kbd>Shift</kbd>+<kbd>Entrée</kbd> exécute une cellule puis passe à la suivante. C'est équivalent à cliquer sur l'icone ▶️ dans la barre de menu du notebook.<br>
- la combinaison de touches <kbd>Alt</kbd>+<kbd>Entrée</kbd> exécute une cellule puis en crée une nouvelle (vide) en dessous.<br>

Pour ajouter une cellule, vous pouvez aussi cliquer sur l'icone ➕ dans la barre de menu du notebook.
- vous pouvez déplacer les cellules en les glissant pour les réorganiser les unes en-dessous des autres.<br>
- vous pouvez ajouter des commentaires, soit en commençant la ligne par un "#" dans une cellule de code (ces lignes ne seront pas éxécutées), soit dans une nouvelle céllule de type Markdown.<br>
- vous sélectionner le type de cellule dans le menu en haut de votre Vnotebbok.<br>
    - "Code" pour entrer des lignes de commande à éxécuter <br>
    - "Markdown pour ajouter du texte balisé qui peut être formatté<br>
- pour modifier une cellule ̀Markdown, double-cliquez dessus<br>
- attention à la casse des caractères pour les cellules de code<br>
- évitez les caractères spéciaux dans les cellules de code<br>
    
<em>  
To make nice html reports with markdown: <a href="https://dillinger.io/" title="dillinger.io">html visualization tool 1</a> or <a href="https://stackedit.io/app#" title="stackedit.io">html visualization tool 2</a>, <a href="https://www.tablesgenerator.com/markdown_tables" title="tablesgenerator.com">to draw nice tables</a>, and the <a href="https://medium.com/analytics-vidhya/the-ultimate-markdown-guide-for-jupyter-notebook-d5e5abf728fd" title="Ultimate guide">Ultimate guide</a>. <br>
Further reading on JupyterLab notebooks: <a href="https://jupyterlab.readthedocs.io/en/latest/user/notebook.html" title="Jupyter Lab">Jupyter Lab documentation</a>.<br>
Here we are using JupyterLab interface implemented as part of the <a href="https://plasmabio.org/" title="plasmabio.org">Plasmabio</a> project led by Sandrine Caburet, Pierre Poulain and Claire Vandiedonck.
</em>    
</div>